In [ ]:
# https://github.com/tensorflow/examples/blob/master/lite/examples/image_classification/raspberry_pi/classify_picamera.py
import cv2
import numpy as np
import tensorflow as tf
import time
import pygame
import time
from gtts import gTTS
import random
from IPython.display import Audio
import speech_recognition as sr

interpreter = tf.lite.Interpreter(model_path='drowsinessDetection.tflite')

ValueError: ignored

In [ ]:
!pip install gtts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install speech_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement speech_recognition (from versions: none)
ERROR: No matching distribution found for speech_recognition


In [ ]:
pygame.mixer.init()
pygame.mixer.music.load('004_b.mp3')

face_cascade_name = 'haarcascade_frontalface_alt.xml'
eyes_cascade_name = 'haarcascade_eye_tree_eyeglasses.xml'

NameError: ignored

In [ ]:
face_cascade = cv2.CascadeClassifier()
eyes_cascade = cv2.CascadeClassifier()

#-- 1. Load the cascades
if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('--(!)Error loading face cascade')
    exit(0)
if not eyes_cascade.load(cv2.samples.findFile(eyes_cascade_name)):
    print('--(!)Error loading eyes cascade')
    exit(0)

In [ ]:
#-- 2. Load TF model
interpreter.allocate_tensors()

SZ = 24
status = 'Awake'
number_closed = 0
closed_limit = 7
show_frame = None
sign = None
color = None

frame_width = 640
frame_height = 480
frame_resolution = [frame_width, frame_height]
frame_rate = 16

In [ ]:
def set_input_tensor(interpreter, image):
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = image


def classify_image(interpreter, image, top_k=1):
  set_input_tensor(interpreter, image)
  interpreter.invoke()
  output_details = interpreter.get_output_details()[0]
  output = np.squeeze(interpreter.get_tensor(output_details['index']))

  # If the model is quantized (uint8 data), then dequantize the results
  if output_details['dtype'] == np.uint8:
    scale, zero_point = output_details['quantization']
    output = scale * (output - zero_point)

  ordered = np.argpartition(-output, top_k)
  return [(i, output[i]) for i in ordered[:top_k]]

In [ ]:
def quiz():
    f = open("quiz.csv")
    lines = f.readlines()
    lines = lines[1:]
    random.shuffle(lines)

    r = sr.Recognizer()
    mic = sr.Microphone()
    
    for line in lines:
        quiz, answer = line.rstrip().split(',')
        correct = None
        while not correct:
            if correct is None:
                kor_wav = gTTS(quiz, lang = 'ko')
                kor_wav.save("kor.wav")
                display(Audio('kor.wav', autoplay=True))
            else:
                kor_wav2 = gTTS("다시 말씀해주세요", lang='ko')
                kor_wav2.save("kor2.wav")
                display(Audio('kor2.wav', autoplay=True))

            ans = None

            while not ans:
                with mic as source :
                    audio = r.listen(source)
                    time.sleep(1)
                    ans = r.recognize_google(audio, language = "ko-KR")
                    time.sleep(1)

            print(ans)
            if ans == "멈춰":
                break
            if answer == ans:

                correct = True
                kor_wav3 = gTTS("정답입니다!", lang='ko')
                kor_wav3.save("kor3.wav")
                display(Audio('kor3.wav', autoplay=True))
            else:
                correct = False
                kor_wav4 = gTTS(f"정답은 {answer}입니다.", lang='ko')
                kor_wav4.save("kor4.wav")
                display(Audio('kor4.wav', autoplay=True))
            time.sleep(1)

In [ ]:
# initialize the camera and grab a reference to the raw camera capture
capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
fps = capture.get(cv2.CAP_PROP_FPS)
# allow the camera to warmup
time.sleep(0.1)
word = None
r = sr.Recognizer()
mic = sr.Microphone()
# capture frames from the camera

while capture.isOpened():
    ret, frame = capture.read()
    start_time = time.time()
    image = cv2.GaussianBlur(frame,(5,5), cv2.BORDER_DEFAULT)
    show_frame = image
    # transform gray image
    height,width = image.shape[:2]
    frame_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.equalizeHist(frame_gray)
    #-- Detect faces
    faces = face_cascade.detectMultiScale(frame_gray)
    with mic as source :
        audio = r.listen(source)
    try:
        word = r.recognize_google(audio, language = "ko-KR")
    except:
        word = None
    if word == "퀴즈":
        quiz()
    else:
        pass

        

    for (x,y,w,h) in faces:
        show_frame = cv2.rectangle(show_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        faceROI = frame_gray[y:y+h,x:x+w]
        #-- In each face, detect eyes
        eyes = eyes_cascade.detectMultiScale(faceROI)
        results = []

        for (x2,y2,w2,h2) in eyes:
            eye_center = (x + x2 + w2//2, y + y2 + h2//2)
            radius = int(round((w2 + h2)*0.25))
            show_frame = cv2.circle(show_frame, eye_center, radius, (0, 255, 255 ), 2)
            eye = faceROI[y2:y2+h2,x2:x2+w2]
            eye = cv2.resize(eye,(SZ,SZ))
            eye= eye/255
            eye=  eye.reshape(SZ,SZ,-1)
            eye = np.expand_dims(eye,axis=0)
            #result = model.predict_classes(eye)
            prediction = classify_image(interpreter, eye)
            result, confidence = prediction[0]
            results.append(result)
        print(results, np.mean(results))
        if( np.mean(results)==1 ):
            color = (0, 255, 0)
            status = 'Awake'
            number_closed = number_closed - 1
            if( number_closed<0 ):
                number_closed = 0
        else:
            color = (0, 0, 255)
            status = 'Sleep'
            number_closed = number_closed + 1
            
        sign = status + ', Sleep count : ' + str(number_closed) + ' / ' + str(closed_limit)
        if( number_closed > closed_limit ):
            show_frame = frame_gray
            # play SOUND
            if(pygame.mixer.music.get_busy()==False):
                pygame.mixer.music.play()
            #while pygame.mixer.music.get_busy() == True:
            #    continue
        
    # show the frame
    cv2.putText(show_frame, sign , (10,height-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    cv2.imshow("Drowsiness Detection", show_frame)
    end_time = time.time()
    process_time = end_time - start_time
    print("=== A frame took {:.3f} seconds".format(process_time))
    # if the `q` key was pressed, break from the loop
    ret, frame = capture.read()
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    ret, frame = capture.read()
    if (type(frame) == type(None)):
        break

    
    if cv2.waitKey(1)& 0xFF == ord('q'):
        break
capture.release()
cv2.destroyAllWindows()

=== A frame took 13.763 seconds
